### COM361 &mdash; Introdução a Otimização &mdash; 2022, Prof. Amit ###

# Gerenciamento de Dívida de Cartão de Crédito #

#### Camila Maia de Carvalho (camilamaia@poli.ufrj.br), Ruth Menezes (ruthmenezesbr@poli.ufrj.br)

*****

### Índice

1. [Introdução](#1.-Introdução)
2. [Modelo Matemático](#2.-Modelo Mathemático)
3. [Solução](#3.-Solução)
4. [Resultados e Discussão](#4.-Resultados-e-discussão)
  4.1. [Subseção Opcional](#4.A.-Acrescente-subseções-se-necessário)
5. [Conclusão](#5.-Conclusão)
6. [Referências bibliográficas] (#6.-Referências bibliográficas)

## 1. Introdução ##

O projeto em questão visa um modelo de otimização de gerenciamento de dívidas de cartão de crédito para que os indivíduos que possuam alguma dívida possam utilizar desse modelo para obter uma solução viável economicamente para saldar suas contas. No caso, o indivíduo em questão possui três cartões de crédito distintos, com diferentes saldos e taxas de juros e uma quantia para saldar suas dívidas de forma parcial. O modelo irá implementar um sistema de controle mensal e por quanto tempo tal modelo irá funcionar.

Dito isso, o modelo funcionará da seguinte forma: 
1. Os três cartões do indivíduo terão saldos $x_1$,$x_2$ e $x_3$ ;
2. As taxas de juros serão de $\alpha_1$ , $\alpha_2$ e $\alpha_3$ ;
3. Conta corrente de saldo $y(k)$, onde k é o instante da dívida;
4. A quantia $u(k)$ é utilizada para saldar a dívida, onde k é o instante.

Observação: Vale ressaltar que vamos considerar para este modelo uma taxa de juros de cheque especial de 6,6% ao mês e como taxa de juros para o país em torno de 1,43% ao mês. 

Logo, $x(k + 1) = (1 + \alpha)x(k) - u(k)$ . 

O projeto em questão tem a sua devida importância, principalmente, quando se trata do cenário atual de endividamento da população brasileira. Segundo o portal de notícias do G1, cerca de 85,3% das famílias brasileiras são afetadas por dívidas de cartão de crédito, sendo este o maior índice quando comparado a outros tipos de dívida, como chegue especial ou crédito consignado. Portanto, é fundamental que os brasileiros tenham ferramentas que os auxiliem em relação a questão do endividamento. 

## 2. Modelo matemático ##

Para o modelo matemático do problema, utilizaremos a equação abaixo: 

$x_i(k + 1) = (1 + \alpha_i)x_i(k) - u_i(k)$

Ou seja, o saldo do cartão i no instante (k + 1) é igual a 1 mais a taxa de juros aplicada sobre o saldo anterior multiplicada pelo saldo no instante K subtraída do pagamento efetuado para quitar o saldo de forma parcial. 

Além disso, pode ser modelada também a conta corrente: 

$y(k+1) = y(k) - u_{total}(k)$

Logo, o saldo da conta corrente no momento $K + 1$ é igual ao saldo da conta no instante $k$ menos o valor retirado da conta para saldar a dívida ($u(k)$). 

Para esse problema também, precisaremos das variáveis de decisão, as quais são representadas pelos pagamentos realizados por mês para saldar a dívida e os saldos das dívidas dos três cartões em cada instante de tempo. 

A seguir, está representada a função quadrático que será utilizada, a qual foi retirada do livro "Introduction to Optimization", de Edwin K. P. Chong e Stanislaw H.:

minimizar 
$\frac{1}{2}\sum_{i=1}^{n}(q(x_i)^2 + r(u_i)^2)$

Bem como aponta o livro utilizado, os parâmetros q e r refletem a prioridade no trade-off entre a redução da dívida e as dificuldades em fazer pagamentos. Quanto mais ansioso o indivíduo estiver para reduzir sua dívida, maior o valor de q em relação a r. Por outro lado, quanto mais relutante em fazer pagamentos, maior o valor de r em relação a q. Com a variação desses parêmtros, é possível ajustar para diferentes indivíduos dependendo da situação financeira da pessoa em questão. 

O problema em questão é do tipo LQR (Linear Quadratic Regulator), o qual se baseia em desigualdades matriciais lineares para sistemas lineares contínuos e invariantes no tempo sujeitos a incertezas politópicas (politopo é a casca convexa de um conjunto finito de vértices) no modelo.

Ouuuu (Ruth, me ajuda a escolher qual das frases é melhor)

O problema em questão é do tipo LQR (Linear Quadratic Regulator), visto que a matriz de ganhos será obtida a partir da minimização de um funcional de custo quadrático. 

Portanto, o modelo proposto é exposto a seguir: 

minimizar 

$\frac{1}{2} \sum_{i=1}^{n} q((x_{1i})^2 + (x_{2i})^2 + (x_{3i})^2) + r((u_{1i})^2 + (u_{2i})^2 + (u_{3i})^2) $

Sujeito a: 

$x_1(k + 1) = (1 + \alpha_1)x_1(k) - u_1(k) \\ $ 
$x_2(k + 1) = (1 + \alpha_2)x_2(k) - u_2(k) \\ $
$x_3(k + 1) = (1 + \alpha_3)x_3(k) - u_3(k) \\ $
$y(k+1) = y(k) - u_{total}(k) \\ $

## 3. Solução ##

Nesta seção, coloque seu código em Julia + JuMP e resolva o problema proposto. Seu código deve ser limpo (não macarrônico!), de fácil leitura, bem comentado e anotado e deve compilar sem erros em Julia 1.x, x$\geq 1$! Não valem códigos em outras linguagens. **Vou rodar seu código para avaliar seu projeto**. Sugiro a utilização de múltiplos blocos de códigos separados por blocos de texto (células Markdown) explicando as várias partes da sua solução. Sugiro também a resolução de várias versões do seu problema, com modelos e hipóteses diferentes.

É permitido chamar pacotes externos, mas evite a utilização de bibliotecas exóticas (pois, em geral, não rodam em todas as versões de Julia, e terei que instalar a mesma versão que você usou, ou rodar na plataforma Google Colab, que gostaria de evitar).

In [ ]:
# Este é um exemplo de um bloco de código
using JuMP, Clp
m = Model(with_optimizer(Clp.Optimizer,LogLevel=0) )
bichos = [:cavalos, :jegues, :cabras]  # estes são os bichos 
@variable(m, x[bichos] >= 0)          # as quantidades de cada um (não podem ser negativas)
@constraint(m, sum(x) <= 10)          # não podemos ter mais de 10 no total.
@objective(m, Max, x[:cavalos])        # queremos maximizar o número de cavalos
optimize!(m)

for i in bichos
    println("O número total de ", i, " é: ", JuMP.value.(x[i]))     # imprime o resultado na tela
end

O número total de cavalos é: 10.0
O número total de jegues é: 0.0
O número total de cabras é: 0.0


**Tenha certeza de que seu código compila corretamente! Rodarei seu código!**

## 4. Resultados e discussão ##

Neste seção, os resultados obtidos serão exibidos e discutidos. Mostre figuras, gráficos, imagens, curvas de compromisso, e o que mais puder melhor ilustrar seus resultados. A discussão deverá explicar o que significam os resultados e como interpretá-los. As limitações da sua abordagem/modelo também devem ser colocadas, bem como uma análise da sensibilidade dos resultados em relação às hipóteses feitas.


Utilize plots (veja exemplos  `PyPlot` [aqui](https://gist.github.com/gizmaa/7214002))

Aqui está um exemplo de uma tabela (em Markdown):

| Tabelas        | São           | Boas  |
| ------------- |:-------------:| -----:|
| col 3 é      | alinhado à direita |\$1600 |
| col 2 é      | centrado      |  \$12 |
| texto | também serve      |   \$1 |

### 4.A. Subseções devem ser utilizadas para organizar seu texto.

#### 4.A.a. ou até subsubseções.

## 5. Conclusão ##

Faça um resumo do que encontrou e dos seus resultados, e fale de pelo menos uma direção na qual  seu trabalho pode ser desenvolvido no futuro, algo que poderia ser interessante em decorrência do seu projeto.


## 6. Referências bibliográficas ##

Livro "An introdution to optimization" 
https://medium.com/@controleavancadoemultivariavel/projeto-de-controladores-%C3%B3timos-lqr-42e16cfe48ce
https://fei.edu.br/sbai/SBAI2007/docs/30870_1.pdf
https://www.bcb.gov.br/estatisticas/reporttxjuros?parametros=tipopessoa:1;modalidade:216;encargo:101
https://www.bcb.gov.br/controleinflacao/historicotaxasjuros